In [1]:
import ray
#ray.shutdown()
#ray.init()

In [2]:
from files import BCTrade_utils
from azureml.core import Workspace,Dataset

ws = Workspace.from_config()
name ='Bitstamp_2'
dataset = Dataset.get_by_name(ws, name=name)
env = BCTrade_utils.BCTrade(dataset.to_pandas_dataframe(),singlerun=False,spreadfactor=0.0)
experiment_name = BCTrade_utils.registerBCTrade(name)
from ray.tune.registry import register_env
def env_creator(env_config):
    return BCTrade_utils.BCTrade(dataset.to_pandas_dataframe())
register_env("BCTrade-Bitstamp_2", env_creator)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [3]:
#datastore = ws.get_default_datastore()
#checkpoint_ds = Dataset.File.from_files(path=[(datastore,'checkpoints_BCTrade-Btc_with_indicators_1641664748_e3baadf3_head/checkpoint_370')])
#checkpoint = checkpoint_ds.as_named_input('test')
#checkpoint.as_mount('xxx')
#checkpoint = '/tmp/checkpoint_370/checkpoint-370'
#Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_100
checkpoint = '/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/'
checkpoint+='logs/PPO/PPO_BCTrade-Bitstamp_1_beeb32bc_2022-03-06_08-59-358hl16iub/checkpoint_351/checkpoint-351'

#checkpoint = 'logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_110/checkpoint-110'

In [4]:
import ray
import ray.rllib.agents.ppo as ppo

#ray.init()
#config = {
  #"callbacks": {
  #  "on_train_result": "<function on_train_result at 0x7fca737a8e18>"
  #},
#  "env": "BCTrade-Btc_with_indicators",
#  "num_gpus": 0,
#  "num_workers": 1
#}

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
#config["eager"] = False
config["env"] = "BCTrade-Bitstamp_2"
new_trainer = ppo.PPOTrainer(config=config,env="BCTrade-Bitstamp_2")
new_trainer


2022-03-06 12:03:20,596	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
(RolloutWorker pid=32708) /anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float64
(RolloutWorker pid=32708)   "Box bound precision lowered by casting to {}".format(self.dtype)
(RolloutWorker pid=32708) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(RolloutWorker pid=32708) Instructions for updating:
(RolloutWorker pid=32708) If using Keras pass *_constraint arguments to layers.
(RolloutWorker pid=32708) 2022-03-06 12:03:45,682	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


PPO

In [5]:
import pickle
with open(checkpoint, "rb") as f:
        extra_data = pickle.load(f)

extra_data['worker']
weights=pickle.loads(extra_data['worker'])
weights['state']['default_policy']
new_trainer.get_policy().set_weights(weights['state']['default_policy'])
#new_trainer.get_policy().set_state(weights['state'])
#new_trainer.restore(checkpoint)
#ray.tune.run(ppo.PPOTrainer, config=config, restore='/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_95fc9bdc_2022-01-10_21-38-0176w5xe80/checkpoint_822')

In [6]:
#obs = env.initialize()
obs = env.reset()
rewardsum=0
c=0
while True:
	action = new_trainer.compute_action(obs, explore=False)
	if obs[0]==1 and action ==2:
		env.render()
	obs, reward, done, _ = env.step(action)
	rewardsum+=reward
	c+=1
	if done:
		break
env.render()
print(rewardsum,rewardsum/c,c)


2022-03-06 12:04:22,888	WARNING deprecation.py:46 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!


1000.0
999.2950526883636
999.8775430489482
999.2511832814308
998.5134998473258
998.4974198790853
996.232021300246
997.6194774844173
997.5906299455842
996.6910141049347
994.5219252008433
996.3139643558645
995.864188221936
995.6890196707848
993.2274563651237
988.6596342421674
987.104580052739
985.7999416279888
987.226641821438
986.9399036902332
988.9326502910309
990.1093772929789
991.0680286637063
993.566358821063
991.7228755815097
989.8336698507459
987.5182076236862
986.5455020305835
986.7357911538016
987.6901988983885
989.9507619281542
987.3390811877072
985.5433430492466
985.6805828573735
985.7777359651259
987.001792914813
989.1992390777551
991.4266257668168
992.7661744234441
994.3908264796213
994.3908264796213
994.5384460906274
994.5907941316963
994.8581817716166
992.6283995243792
992.1098204735508
993.2643136331726
993.2580224052089
993.2287112712337
994.3791720672141
994.212544266499
993.6805526717741
993.6805526717741
990.6984574591629
990.3560838122262
990.2753713055408
990.554302